In [1]:
import pandas as pd # для работы с данными
import numpy as np  # для работы с данными
from sklearn.linear_model import LinearRegression # для построения модели
from sklearn.model_selection import train_test_split # для разделение выборки на тренировочную и тестовую
from sklearn.feature_selection import RFE # для отбора признаков
from sklearn.feature_selection import SelectKBest, f_regression # для отбора признаков
from sklearn import metrics # для расчета метрик

In [92]:
df = pd.read_excel('data/data_ford_price.xlsx')

In [93]:
# заменим пропуски в поле weather на средним зачение по полю
df['weather'] = df['weather'].fillna(np.round(np.mean(df['weather']), 2))

In [94]:
# проверим наличие пропусков
df.isna().mean() == 0

price            True
year             True
condition        True
cylinders        True
odometer         True
title_status     True
transmission     True
drive           False
size            False
lat              True
long             True
weather          True
dtype: bool

In [95]:
# отберем признаки с числовыми данными
df_new = df[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]

In [96]:
# раздалим целевой признак и факторы
y = df_new['price'] 
X = df_new.drop(columns='price')

In [97]:
# разделим данные на тренировочные и тестовые
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

##### Применим Метод рекурсивного исключения признаков (RFE)

In [98]:
# применим RFE для подбора признаков 
model_rfe = LinearRegression()
selector = RFE(model_rfe, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [106]:
list_rfe = selector.get_feature_names_out().tolist()
list_rfe

['year', 'cylinders', 'lat']

По итогам методам RFE 3 наиболее значимые поля: year, cylinders, lat.

In [100]:
# отберем признаки и на них обучим модель
X_train_rfe = X_train[list_rfe]
X_valid_rfe = X_valid[list_rfe]

In [101]:
# обучим модель, сделаем предсказание и подсчитаем метрики МАЕ и R2 на тестовой выборке
model_rfe.fit(X_train_rfe, y_train)
y_valid_pred = model_rfe.predict(X_valid_rfe)

print('Valid MAE on RFE:{:.3f}'.format(metrics.mean_absolute_error(y_valid, y_valid_pred)))
print('Valid R2 on RFE:{:.3f}'.format(metrics.r2_score(y_valid, y_valid_pred)))

Valid MAE on RFE:5513.684
Valid R2 on RFE:0.386


##### Примеим метод выбора признаков на основе фильтров (Метод выбора k лучших переменных)

In [102]:
selector_kbest = SelectKBest(f_regression, k=3)
selector_kbest.fit(X_train, y_train)
 
selector_kbest.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [105]:
list_kbest = selector_kbest.get_feature_names_out().tolist()
list_kbest

['year', 'cylinders', 'odometer']

По итогам методам SelectKBest 3 наиболее значимые поля: year, cylinders, odometer.

In [107]:
# отберем признаки и на них обучим модель
X_train_kbest = X_train[list_kbest]
X_valid_kbest = X_valid[list_kbest]

In [109]:
# обучим модель, сделаем предсказание и подсчитаем метрики МАЕ и R2 на тестовой выборке
model_rfe.fit(X_train_kbest, y_train)
y_valid_pred = model_rfe.predict(X_valid_kbest)

print('Valid MAE on KBEST:{:.3f}'.format(metrics.mean_absolute_error(y_valid, y_valid_pred)))
print('Valid R2 on KBEST:{:.3f}'.format(metrics.r2_score(y_valid, y_valid_pred)))

Valid MAE on KBEST:5120.182
Valid R2 on KBEST:0.413


**Вывод:** по итогам анализа метод RFE показал более лучший результат на метрике коэффициент детерминации R2 (0.386) в отличие от метода SELECTKBEST (0.413). Однако при этом метод RFE по метрике средняя абсолютная ошибка показал результат хуже.